In [1]:
import pandas as pd

<h1>UserForGenre</h1>

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [2]:
# Cargamos los archivos CSV a utilizar:
games_df = pd.read_csv(r"..\Archivos Producidos\games.csv")
reviews_df = pd.read_csv(r"..\Archivos Producidos\reviews.csv")
users_df = pd.read_csv(r"..\Archivos Producidos\users.csv")

# Convertimos algunos tipos de columna que no se cargaron correctamente:
reviews_df["item_id"] = reviews_df["item_id"].astype("Int64")
reviews_df["year"] = reviews_df["year"].astype("Int64")

In [3]:
games_df.dtypes

app_name                     object
release_year                  int64
item_id                       int64
developer                    object
Action                        int64
Adventure                     int64
Animation &amp; Modeling      int64
Audio Production              int64
Casual                        int64
Design &amp; Illustration     int64
Early Access                  int64
Education                     int64
Free to Play                  int64
Indie                         int64
Massively Multiplayer         int64
Photo Editing                 int64
RPG                           int64
Racing                        int64
Simulation                    int64
Software Training             int64
Sports                        int64
Strategy                      int64
Utilities                     int64
Video Production              int64
Web Publishing                int64
dtype: object

In [4]:
reviews_df.dtypes

user_id               object
item_id                Int64
recommend               bool
review                object
year                   Int64
sentiment_analysis     int64
dtype: object

In [5]:
users_df.dtypes

user_id     object
playtime     int64
item_id      int64
dtype: object

Ya cargados los archivos, comenzamos con las transformaciones:

In [6]:
# Unimos las columnas que nos serán de utilidad:
df_u_r = pd.merge(users_df[["user_id", "item_id", "playtime"]], reviews_df[["user_id", "year"]], on="user_id")
df_u_r.head()

,user_id,item_id,playtime,year
0,76561197970982479,10,6,2011
1,76561197970982479,10,6,2011
2,76561198089393905,4000,47269,2014
3,76561198156664158,4000,269,<NA>
4,76561198156664158,4000,269,<NA>


In [7]:
# Agrupamos por usuario y año, y sumamos las horas jugadas:
df_sum_usertime = df_u_r.groupby(["user_id", "item_id", "year"])["playtime"].sum().reset_index()
df_sum_usertime.head()

,user_id,item_id,year,playtime
0,--000--,4000,2014,925
1,--ace--,4000,2014,160
2,--ionex--,10180,2015,105
3,-kainey9777,339610,2015,0
4,0-3-0,1610,2014,26


In [8]:
# Unimos el dataframe anterior con los dummies de géneros:
df_userforgenre = pd.merge(df_sum_usertime, games_df[['item_id', 'Action',
       'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing']], on="item_id")

df_userforgenre.head()

,user_id,item_id,year,playtime,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,--000--,4000,2014,925,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,--ace--,4000,2014,160,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,01001000-01101001,4000,2014,2124,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,011111135489484797,4000,2013,15440,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,01221733,4000,2015,3646,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [9]:
# Utilizamos melt para convertir el DataFrame de ancho a largo:
df_melted = pd.melt(df_userforgenre, id_vars=["user_id", "item_id", "year", "playtime"], 
                    var_name="genre", value_name="has_genre")

# Filtramos las filas donde el género está presente (has_genre == 1):
df_filtered = df_melted[df_melted["has_genre"] == 1]

# Agrupamos por "user_id", "release_year" y "genre", y sumamos las horas jugadas:
df_userforgenre_final = df_filtered.groupby(["user_id", "year", "genre"], as_index=False)["playtime"].sum()

# Capitalizamos el nombre del género:
df_userforgenre_final["genre"] = df_userforgenre_final["genre"].str.replace('&amp;', '&').apply(str.capitalize)

print(df_userforgenre_final)

             user_id  year       genre  playtime
0            --000--  2014       Indie       925
1            --000--  2014  Simulation       925
2            --ace--  2014       Indie       160
3            --ace--  2014  Simulation       160
4          --ionex--  2015      Action       105
...              ...   ...         ...       ...
17039      _spathis_  2013      Action       246
17040      _spathis_  2014      Action       123
17041  _steelphantom  2014      Action      1211
17042  _steelphantom  2015      Action      1211
17043       _useless  2015         Rpg       843

[17044 rows x 4 columns]


Desarrollamos la función:

In [10]:
# Creamos la función UserForGenre:
def UserForGenre(genero):

    # Verificamos si la variable introducida es una cadena de texto:
    if type(genero) != str: return f"El género '{genero}' no existe."
    else:

        # Capitalizamos la cadena de texto:
        genero = genero.strip().capitalize()
    
        # Comprobamos si el género existe en el DataFrame:
        if genero not in df_userforgenre_final["genre"].tolist():
            return f"El género '{genero}' no existe o no tiene horas de juego."

        # En caso de que el género introducido si tenga horas de juego:
        else:

            # Filtramos por género, ordenamos con orden de años descendente:
            df_genero = df_userforgenre_final[df_userforgenre_final["genre"] == genero].sort_values(by="year", ascending=False)

            # Agrupamos haciendo suma de las horas jugadas según usuario:
            df_genero_grouped = df_genero.groupby(["user_id", "genre"], as_index=False)["playtime"].sum()

            # Hallamos el índice del mayor tiempo de juego:
            indice_max_playtime = df_genero_grouped["playtime"].idxmax()

            # Hallamos el usuario con dicho índice:
            user = df_genero_grouped.loc[indice_max_playtime]["user_id"]

            # Buscamos el historial de juego del usuario:
            historial = df_genero[df_genero["user_id"] == user]

            # Creamos la variable que utilizaremos como respuesta final:
            diccionario = {f"Usuario con más horas jugadas para Género {genero}" : user,
                        "Horas jugadas":[]}

            # Rellenamos esa variable:
            for index, row in historial.iterrows():
                diccionario["Horas jugadas"].append({"Año": row["year"], "Horas": row["playtime"]})

            # Devolvemos la respuesta final:
            return diccionario

In [11]:
# Probamos la función
print(UserForGenre("Simulation  "))
print(UserForGenre("acTion"))
print(UserForGenre("Acción"))
print(UserForGenre(123))
print(UserForGenre("Rpg"))
print(UserForGenre("  photo editing"))

{'Usuario con más horas jugadas para Género Simulation': '76561198047208774', 'Horas jugadas': [{'Año': 2015, 'Horas': 231334}, {'Año': 2012, 'Horas': 231334}]}
{'Usuario con más horas jugadas para Género Action': 'Jantoz', 'Horas jugadas': [{'Año': 2014, 'Horas': 894792}]}
El género 'Acción' no existe o no tiene horas de juego.
El género '123' no existe.
{'Usuario con más horas jugadas para Género Rpg': '76561198166194627', 'Horas jugadas': [{'Año': 2014, 'Horas': 187157}]}
El género 'Photo editing' no existe o no tiene horas de juego.


Guardamos el DataFrame utilizado:

In [12]:
# Guardamos el dataframe "df_userforgenre_final":
df_userforgenre_final.to_csv(r"..\Archivos Producidos\Data para Endpoints\df_userforgenre_final.csv", index=False)